
## Spin glass $N=14$
 

In [ ]:
import itertools
import os

import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

In [ ]:
n = 14 
circ_depths = np.asarray([1,2])
algs = np.asarray(["qaoa", "qaoa+", "vqe"])
seeds = np.arange(1, 31)

#model_info = [(n, algo, p, seed) for p in circ_depths for algo in algs for seed in seeds]

In [ ]:
jobs_df = pd.read_csv("data_n14_spinglass.csv")

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(24, 5))
for shot, data in jobs_df.groupby('shots'):
    if shot != 512:
        continue

    boxplot_data = []
    for it in np.arange(5,40,5):
        boxplot_data.append(data[(jobs_df['algo'] == 'qaoa') & (data['iter'] == it)]['p_succ'])
    ax1.boxplot(boxplot_data)
    ax1.set_title(f"QAOA shots={shot}")
    ax1.set_xticks(np.arange(8), np.arange(0,40,5))
    ax1.set_xlabel("iterations")
    ax1.set_ylabel(r"$F('success')$")

    boxplot_data = []
    for it in np.arange(5,40,5):
        boxplot_data.append(data[(jobs_df['algo'] == 'qaoa+') & (data['iter'] == it)]['p_succ'])
    ax2.boxplot(boxplot_data)
    ax2.set_title(f"QAOA+ shots={shot}")
    ax2.set_xticks(np.arange(8), np.arange(0,40,5))
    ax2.set_ylim(0,1)
    ax2.set_xlabel("iterations")
    ax2.set_ylabel(r"$F('success')$")
 
    boxplot_data = []
    for it in np.arange(5,40,5):
        boxplot_data.append(data[(jobs_df['algo'] == 'vqe') & (data['iter'] == it)]['p_succ'])
    ax3.boxplot(boxplot_data)
    ax3.set_title(f"VQE shots={shot}")
    ax3.set_xticks(np.arange(8), np.arange(0,40,5))
    ax3.set_ylim(0,1)
    ax3.set_xlabel("iterations")
    ax3.set_ylabel(r"$F('success')$")

plt.savefig("boxplot_shot512.png")

In [ ]:
for shot, data in jobs_df[(jobs_df['algo'] == 'qaoa') & (jobs_df['iter'] < 37)].groupby('shots'):
    x_axes = data[data['seed'] == 1]['t']*data[data['seed'] == 1]["circ_depth"]
    y_axes = data.groupby(['iter'])['p_succ'].mean()
    error = data.groupby(['iter'])['p_succ'].std()

    plt.errorbar(x_axes, y_axes, yerr=data.groupby(['iter'])['p_succ'].std(), marker="o", label=f"{shot}")
    plt.fill_between(x_axes, data.groupby(['iter'])['p_succ'].min(), data.groupby(['iter'])['p_succ'].max(), alpha=0.3)

plt.plot(np.arange(0,2**(3+n),2), 1 - (1-1/(2**n))**np.arange(0,2**(3+n),2), '--', color="black", label="Random sampling")
plt.xscale("log", base=2)

plt.xlabel("iterations x shots x depth")
plt.ylabel(r"$F('success')$")

plt.title(f"$N=14$ QAOA $p=2$")
plt.legend(title='Shots')
plt.savefig("n14_spinglass_qaoap2.png")
    #print(data[data['seed'] == 1]['t'])

In [ ]:
for shot, data in jobs_df[(jobs_df['algo'] == 'qaoa+')].groupby('shots'):
    plt.errorbar(data[data['seed'] == 1]['t'], data.groupby(['iter'])['p_succ'].mean(), yerr=data.groupby(['iter'])['p_succ'].std(), marker="o", label=f"{shot}")

plt.plot(np.arange(0,2**(3+n),2), 1 - (1-1/(2**n))**np.arange(0,2**(3+n),2), '--', color="black", label="Random sampling")
plt.xscale("log", base=2)

plt.xlabel("iterations x shots x depth")
plt.ylabel(r"$F('success')$")

plt.title(f"$N=14$ QAOA+ $p=1$")
plt.legend(title='Shots')
plt.savefig("n14_spinglass_qaoap+2.png")


In [ ]:
for shot, data in jobs_df[(jobs_df['algo'] == 'vqe')].groupby('shots'):
    plt.errorbar(data[data['seed'] == 1]['t'], data.groupby(['iter'])['p_succ'].mean(), yerr=data.groupby(['iter'])['p_succ'].std(), marker="o", label=f"{shot}")

plt.plot(np.arange(0,2**(3+n),2), 1 - (1-1/(2**n))**np.arange(0,2**(3+n),2), '--', color="black", label="Random sampling")
plt.xscale("log", base=2)

plt.xlabel("iterations x shots x depth")
plt.ylabel(r"$F('success')$")

plt.title(f"$N=14$ VQE $p=1$")
plt.legend(title='Shots')
plt.savefig("n14_spinglass_vqep1.png")



## Spin glass $4 \leq N \leq 12$

In [ ]:
import itertools
import os

import numpy as np
import pandas as pd
import json
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
jobs_df = pd.read_csv("dataspinglass-COMPLETE.csv", index_col=False)

In [ ]:
print(jobs_df.head(10))
# uncomment to remove the unnamed column
#jobs_df.drop('Unnamed: 0', axis=1, inplace=True)

# seeds = np.arange(1,31)
# qubits=14
# print(jobs_df[(jobs_df['circ_depth'] == 1) & (jobs_df['iter'] % 5 != 0) & (jobs_df['iter'] != 1)]) 

# indexes = jobs_df[(jobs_df['circ_depth'] == 1) & (jobs_df['iter'] > 25)].index
# jobs_df.drop(indexes, inplace=True)

# print(jobs_df[(jobs_df['circ_depth'] == 1) & (jobs_df['iter'] % 5 != 0) & (jobs_df['iter'] != 1)]) 

jobs_df.to_csv(f"dataspinglass.csv")

In [ ]:
depth = 4

fig = plt.figure(constrained_layout=True, figsize=(16,10))
subplots = fig.subplots(2,3)
for (qubits, data), subplot in zip(jobs_df[(jobs_df['algo'] == 'qaoa') & (jobs_df['circ_depth'] == depth)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        x_axes = dat[dat['seed'] == 1]['t']#*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat.groupby(['iter'])['p_succ'].median()
        error = [
            (dat.groupby(['iter'])['p_succ'].median() - dat.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
            (dat.groupby(['iter'])['p_succ'].quantile(0.75) - dat.groupby(['iter'])['p_succ'].median()).tolist(),
            ]

        subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
        subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black", label="Random sampling")
    subplot.set_xscale("log", base=2)

    subplot.set_xlabel(r"$n_{calls}$")
    subplot.set_ylabel(r"$F_{succ}$")

    subplot.set_title(f"N={qubits} QAOA p={depth}")
    subplot.legend(title='Shots')

#plt.savefig('spinglass_all.png')
    # plt.show()

In [ ]:
# jobs_correct = pd.read_csv("data-n4-8_spinglass-PART.csv")

# fig = plt.figure(constrained_layout=True, figsize=(12,5))
# subplots = fig.subplots(1,2)
# for (qubits, data), subplot in zip(jobs_correct[(jobs_correct['algo'] == 'qaoa') & (jobs_correct['circ_depth'] == 1) & (jobs_correct['iter'] <= 25)].groupby('qubits'), subplots):
#     for shot, dat in data.groupby('shots'):
#         x_axes = dat[dat['seed'] == 1]['t']*dat[dat['seed'] == 1]["circ_depth"]
#         y_axes = dat.groupby(['iter'])['p_succ'].median()
#         error = [
#             (dat.groupby(['iter'])['p_succ'].median() - dat.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
#             (dat.groupby(['iter'])['p_succ'].quantile(0.75) - dat.groupby(['iter'])['p_succ'].median()).tolist(),
#             ]
#         subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
#         subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].min(), dat.groupby(['iter'])['p_succ'].max(), alpha=0.3)

#     subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black", label="Random sampling")
#     subplot.set_xscale("log", base=2)

#     subplot.set_xlabel("iterations x shots x depth")
#     subplot.set_ylabel(r"$F('success')$")

#     subplot.set_title(f"N={qubits} QAOA p=1")
#     subplot.legend(title='Shots')

# plt.savefig('spinglass_correct.png')

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

min_qubits, max_qubits = 6, 14
min_qubits_toplot, max_qubits_toplot = 9, 16
qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)


fig = plt.figure()

plt.plot(np.arange(min_qubits, max_qubits+2), fun_exp(np.arange(min_qubits, max_qubits+2), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

p_tresholds = [0.9, 0.75, 0.5, 0.25]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
for p, color in zip(p_tresholds, colors):
    median_for_seed = jobs_df[(jobs_df['algo'] == 'qaoa') & (jobs_df['circ_depth'] == depth) & (jobs_df['qubits'] >= min_qubits)].groupby(['t', 'qubits', 'circ_depth'], as_index=False).agg(avg_p_succ = ("p_succ", "median"))
    gt_p = median_for_seed[median_for_seed['avg_p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

    popt, _ = optimize.curve_fit(fun_lin, gt_p['qubits'][-3:], np.log2(gt_p['t'][-3:]))
    plt.plot(qubits_toplot, fun_exp(qubits_toplot, 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    
    plt.plot(gt_p['qubits'], gt_p['t'], "o", label=r"$F_{succ}$"+f"={p}", color=color)

    plt.yscale('log', base=2)
    plt.ylabel(r"$N_{calls}$")
    plt.xlabel(r"$L$")
    plt.xticks(np.arange(min_qubits, max_qubits+1, 2))

plt.legend(loc='upper left', title=f"QAOA p={depth}", fontsize="x-small")

plt.title(r"Fit with $a \cdot 2^{b \cdot L}$")

#plt.savefig(f"scale_spinglass-{min_qubits}.png")

In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

min_qubits, max_qubits = 4, 8

fig = plt.figure()

plt.plot(np.arange(min_qubits, max_qubits+1), fun_exp(np.arange(min_qubits, max_qubits+1), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

p_tresholds = [0.99, 0.9, 0.75, 0.5, 0.25]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'cornflowerblue' , 'blue', 'navy']
for p, color in zip(p_tresholds, colors):
    median_for_seed = jobs_correct[(jobs_correct['algo'] == 'qaoa') & (jobs_correct['circ_depth'] == 1)].groupby(['t', 'qubits', 'circ_depth'], as_index=False).agg(avg_p_succ = ("p_succ", "median"))
    gt_p = median_for_seed[median_for_seed['avg_p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

    popt, _ = optimize.curve_fit(fun_lin, gt_p['qubits'], np.log2(gt_p['t']*median_for_seed['circ_depth'][0]))
    plt.plot(gt_p['qubits'], fun_exp(gt_p['qubits'], 2**popt[0], popt[1]), "-", label=f"a={2**popt[0]:.2f}, b={popt[1]:.2f}", color=color)
    
    plt.plot(gt_p['qubits'], gt_p['t']*median_for_seed['circ_depth'][0], "o", label=f"F('succ')={p}", color=color)

    plt.yscale('log', base=2)
    plt.ylabel(r"$t$ (shots x iterations x circ_depth)")
    plt.xlabel(r"$N$")

plt.legend(loc='lower right', title=r"QAOA $p=1$", fontsize="x-small")

plt.title(r"Spin Glass fit with $a \cdot 2^{b \cdot N}$")

In [ ]:
depth = 1

fig = plt.figure(constrained_layout=True, figsize=(16,10))
subplots = fig.subplots(2,3)
for (qubits, data), subplot in zip(jobs_df[(jobs_df['algo'] == 'qaoa') & (jobs_df['circ_depth'] == depth)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        x_axes = dat[dat['seed'] == 1]['t']*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat.groupby(['iter'])['p_succ'].median()
        error = [
            (dat.groupby(['iter'])['p_succ'].median() - dat.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
            (dat.groupby(['iter'])['p_succ'].quantile(0.75) - dat.groupby(['iter'])['p_succ'].median()).tolist(),
            ]

        subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
        subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black", label="Random sampling")
    subplot.set_xscale("log", base=2)

    subplot.set_xlabel("iterations x shots x depth")
    subplot.set_ylabel(r"$F('success')$")

    subplot.set_title(f"N={qubits} QAOA p={depth}")
    subplot.legend(title='Shots')


## For paper

These lines MUST be clean

In [ ]:
import itertools
import os

import numpy as np
import pandas as pd
import json
from scipy import optimize
import matplotlib.pyplot as plt


### Figure 2


In [ ]:
ferro_list = [pd.read_csv(f"data_n{n}-test.csv", index_col=0) for n in range(4,22,2)]
ferro_df = pd.concat(ferro_list, ignore_index=True)
ferro_df.head()

In [ ]:
spinglass = pd.read_csv("dataspinglass.csv", index_col=0)
spinglass.head()

In [ ]:
spinglass16 = pd.read_csv("dataspinglass-n16.csv", index_col=0)
#print(spinglass16)
spinglass_df = pd.concat([spinglass, spinglass16], ignore_index=True)

In [ ]:
# import matplotlib

# matplotlib.rcParams["font.size"] = 12

SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 18

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

fig = plt.figure(constrained_layout=True, figsize=(16,10))
subplots = fig.subplots(2,3)

for i, subplot in enumerate(subplots.ravel()):
    if i == 0:
        qubits = 8
        depth = 2
        max_shots = 128
        for shot, data in ferro_df[(ferro_df['algo'] == 'qaoa') & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] == qubits) & (ferro_df['shots'] < max_shots)].groupby('shots'):
            x_axes = data['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data['p_succ']
            subplot.plot(x_axes, y_axes, "-o", label=f"{shot}")

        subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Ferro " + r"QAOA $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$', loc="upper left")
        subplot.text(2**12, 0.1, "(a)")

    if i == 1:
        qubits = 14
        depth = 2
        for shot, data in ferro_df[(ferro_df['algo'] == 'qaoa') & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] == qubits)].groupby('shots'):
            x_axes = data['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data['p_succ']
            subplot.plot(x_axes, y_axes, "-o", label=f"{shot}")

        subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Ferro " + r"QAOA $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$')
        subplot.text(2**18, 0.1, '(b)')

    if i == 2:
        min_qubits, max_qubits = 4, 18
        min_qubits_toplot, max_qubits_toplot = 7, 19
        qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)
        
        depth1 = 2
        depth2 = 4
        plots1, plots2 =[], []

        subplot.plot(np.arange(min_qubits, max_qubits-4), fun_exp(np.arange(min_qubits, max_qubits-4), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

        p_tresholds = [0.9, 0.75, 0.5, 0.25]
        colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
        for p, color in zip(p_tresholds, colors):
            data1 = ferro_df[(ferro_df['algo'] == 'qaoa') & (ferro_df['circ_depth'] == depth1)]
            data2 = ferro_df[(ferro_df['algo'] == 'qaoa') & (ferro_df['circ_depth'] == depth2)]
    
            gt_p1 = data1[data1['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
            gt_p2 = data2[data2['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
            
            popt1, _ = optimize.curve_fit(fun_lin, gt_p1['qubits'], np.log2(gt_p1['t']))
            popt2, _ = optimize.curve_fit(fun_lin, gt_p2['qubits'], np.log2(gt_p2['t']))

            plot1, = subplot.plot(gt_p1['qubits'], gt_p1['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", color=color)
            plots1.append(plot1)
            plot2, = subplot.plot(gt_p2['qubits'], gt_p2['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", color=color, fillstyle='none')
            plots2.append(plot2)

            plot1,  = subplot.plot(qubits_toplot, fun_exp(qubits_toplot, 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, k={popt1[1]:.2f}", color=color)
            plot2, = subplot.plot(qubits_toplot, fun_exp(qubits_toplot, 2**popt2[0], popt2[1]), "--", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=color)
            plots1.append(plot1)
            plots2.append(plot2)

        subplot.set_yscale('log', base=2)
        subplot.set_ylabel(r"$n^{\ast}_{\mathrm{calls}}$")
        subplot.set_xlabel(r"$L$")
        subplot.set_xticks(np.arange(min_qubits, max_qubits+1, 2))

        first_legend = subplot.legend(handles=plots1, title=r"QAOA $d=$"+f"{depth1}", loc='upper left', fontsize="x-small")
        subplot.add_artist(first_legend)

        subplot.legend(handles=plots2, loc='lower right', title=r"QAOA $d=$"+f"{depth2}", fontsize="x-small")

        subplot.set_title(r"Scaling and fit with $a \cdot 2^{kL}$", fontsize=14)
       
        subplot.text(12, 2**2, '(c)')

    if i == 3:
        depth = 2
        qubits = 8
        for shot, data in spinglass_df[(spinglass_df['algo'] == 'qaoa') & (spinglass_df['circ_depth'] == depth) & (spinglass_df['qubits'] == qubits)].groupby('shots'):
            x_axes = data[data['seed'] == 1]['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data.groupby(['iter'])['p_succ'].median()
            error = [
                (data.groupby(['iter'])['p_succ'].median() - data.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
                (data.groupby(['iter'])['p_succ'].quantile(0.75) - data.groupby(['iter'])['p_succ'].median()).tolist(),
                ]
            subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
            #subplot.fill_between(x_axes, data.groupby(['iter'])['p_succ'].quantile(0.95), data.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

        subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Disordered " + r"QAOA $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$')

        subplot.text(2**12, 0.1, '(d)')

    if i == 4:
        qubits = 14
        depth = 2

        for shot, data in spinglass_df[(spinglass_df['algo'] == 'qaoa') & (spinglass_df['circ_depth'] == depth) & (spinglass_df['qubits'] == qubits)].groupby('shots'):
            x_axes = data[data['seed'] == 1]['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data.groupby(['iter'])['p_succ'].median()
            error = [
                (data.groupby(['iter'])['p_succ'].median() - data.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
                (data.groupby(['iter'])['p_succ'].quantile(0.75) - data.groupby(['iter'])['p_succ'].median()).tolist(),
                ]
            subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
            #subplot.fill_between(x_axes, data.groupby(['iter'])['p_succ'].quantile(0.95), data.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

        subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Disordered " + r"QAOA $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$')

        subplot.text(2**18, 0.1, '(e)')

    if i == 5:
        min_qubits, max_qubits = 6, 16
        min_qubits_toplot, max_qubits_toplot = 9, 18
        qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)

        depth1 = 2
        depth2 = 4
        plots1, plots2 = [], []
        
        subplot.plot(np.arange(min_qubits, max_qubits_toplot+1), fun_exp(np.arange(min_qubits, max_qubits_toplot+1), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

        p_tresholds = [0.9, 0.75, 0.5, 0.25]
        colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
        for j, (p, color) in enumerate(zip(p_tresholds, colors)):
            median_for_seed1 = spinglass_df[(spinglass_df['algo'] == 'qaoa') & (spinglass_df['circ_depth'] == depth1) & (spinglass_df['qubits'] >= min_qubits) & (spinglass_df['qubits'] <= max_qubits)].groupby(['t', 'qubits', 'circ_depth'], as_index=False).agg(avg_p_succ = ("p_succ", "median"))
            median_for_seed2 = spinglass_df[(spinglass_df['algo'] == 'qaoa') & (spinglass_df['circ_depth'] == depth2) & (spinglass_df['qubits'] >= min_qubits)].groupby(['t', 'qubits', 'circ_depth'], as_index=False).agg(avg_p_succ = ("p_succ", "median"))
            
            gt_p1 = median_for_seed1[median_for_seed1['avg_p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
            gt_p2 = median_for_seed2[median_for_seed2['avg_p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

            popt1, _ = optimize.curve_fit(fun_lin, gt_p1['qubits'][-4:], np.log2(gt_p1['t'][-4:]))
            plot1, = subplot.plot(gt_p1['qubits'], gt_p1['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", color=color)
            plots1.append(plot1)
            plot1, = subplot.plot(qubits_toplot, fun_exp(qubits_toplot, 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, k={popt1[1]:.2f}", color=color)
            plots1.append(plot1)

            popt2, _ = optimize.curve_fit(fun_lin, gt_p2['qubits'][-3:], np.log2(gt_p2['t'][-3:]))
            plot2, = subplot.plot(gt_p2['qubits'], gt_p2['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", fillstyle='none', color=colors[j+5])
            plots2.append(plot2)
            plot2, = subplot.plot(qubits_toplot, fun_exp(qubits_toplot, 2**popt2[0], popt2[1]), "--", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=colors[j+5])
            plots2.append(plot2)

        subplot.set_yscale('log', base=2)
        subplot.set_ylabel(r"$n^{\ast}_{\mathrm{calls}}$")
        subplot.set_xlabel(r"$L$")
        subplot.set_xticks(np.arange(min_qubits, max_qubits+1, 2))

        first_legend = subplot.legend(handles=plots1, title=r"QAOA $d=$"+f"{depth1}", loc='upper left', fontsize="x-small")
        subplot.add_artist(first_legend)

        subplot.legend(handles=plots2, loc='lower right', title=r"QAOA $d=$"+f"{depth2}", fontsize="x-small")

        subplot.set_title(r"Scaling and fit with $a \cdot 2^{kL}$", fontsize=14)
       
        subplot.text(13, 2**5.1, '(f)')

plt.savefig("img/fig2.png", dpi=300)


### Figure 1

In [ ]:
noise_df = pd.read_csv("datanoise.csv", index_col=0)
noise_df.head()

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 18

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title


def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

fig = plt.figure(constrained_layout=True, figsize=(16,5))
subplots = fig.subplots(1,3)
plots1, plots2 = [], [] 

for i, subplot in enumerate(subplots.ravel()):
    if i == 0:
        qubits = 8
        depth = 2
        max_shots = 1024
        algo = 'vqe'
        for shot, data in ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] == qubits) & (ferro_df['shots'] < max_shots)].groupby('shots'):
            x_axes = data['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data['p_succ']
            subplot.plot(x_axes, y_axes, "-o", label=f"{shot}")

        subplot.plot(np.arange(0,2**(6+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(6+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Ferro " + r"VQE $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$')

        subplot.text(2**12, 0.1, '(a)')

    if i == 1:
        qubits = 14
        depth = 2
        algo = 'vqe'

        inplot = fig.add_axes([.466, .61, .09, .28])
        xmin, xmax = 25000,300000
        ymin, ymax = 0.75, 1.02

        for shot, data in ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] == qubits)].groupby('shots'):
            x_axes = data['t'] #*dat[dat['seed'] == 1]["circ_depth"]
            y_axes = data['p_succ']
            subplot.plot(x_axes, y_axes, "-o", label=f"{shot}")
            inplot.plot(x_axes, y_axes, "-o")


        subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
        subplot.set_xscale("log", base=2)

        subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
        subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

        subplot.set_title(f"L={qubits} Ferro " + r"VQE $d=$" + f"{depth}", fontsize=14)
        subplot.legend(title=r'Shots $M$')
        
        inplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")

        inplot.set_xlim(xmin, xmax)
        inplot.set_ylim(ymin, ymax)

        inplot.set_xticklabels([r"$2.5 \cdot 10^4$", r"$10^5$", r"$2 \cdot 10^5$", r"$3 \cdot 10^5$"], fontsize="small")
        inplot.set_yticklabels([0.75, 0.8, 0.9, 1.0], fontsize="small")
        inplot.tick_params(  
            axis='both',   # changes apply to both axis
            which='both',      # both major and minor ticks are affected
            bottom=True,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            left = True,
            right=False,
            labelleft=True,
            labelbottom=True,
        )

        subplot.text(2**18, 0.1, '(b)')

    if i == 2:
        min_qubits, max_qubits = 4, 20
        min_qubits_toplot, max_qubits_toplot = 6, 22
        qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)

        depth1 = 2
        depth2 = 2

        maxiter = 40 
        
        subplot.plot(np.arange(min_qubits, max_qubits_toplot), fun_exp(np.arange(min_qubits, max_qubits_toplot), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

        p_tresholds = [0.9, 0.75, 0.5, 0.25]
        colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
        for j, (p, color) in enumerate(zip(p_tresholds, colors)):
            data1 = ferro_df[(ferro_df['algo'] == 'vqe') & (ferro_df['circ_depth'] == depth1) & (ferro_df['qubits'] >= min_qubits) & (ferro_df['iter'] < maxiter)]
            data2 = noise_df[(noise_df['algo'] == 'vqe') & (noise_df['circ_depth'] == depth2) & (noise_df['qubits'] >= min_qubits)]
            
            gt_p1 = data1[data1['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
            gt_p2 = data2[data2['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

            popt1, _ = optimize.curve_fit(fun_lin, gt_p1['qubits'][-4:], np.log2(gt_p1['t'][-4:]))
            plot1, = subplot.plot(gt_p1['qubits'], gt_p1['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", color=color)
            plots1.append(plot1)
            plot1, = subplot.plot(gt_p1['qubits'][1:], fun_exp(gt_p1['qubits'][1:], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, k={popt1[1]:.2f}", color=color)
            plots1.append(plot1)

            popt2, _ = optimize.curve_fit(fun_lin, gt_p2['qubits'][-4:], np.log2(gt_p2['t'][-4:]))
            plot2, = subplot.plot(gt_p2['qubits'], gt_p2['t'], "o", label=r"$F_{\mathrm{succ}}$"+f"={p}", fillstyle='none', color=colors[j+5])
            plots2.append(plot2)
            plot2, = subplot.plot(gt_p2['qubits'][1:], fun_exp(gt_p2['qubits'][1:], 2**popt2[0], popt2[1]), "--", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=colors[j+5])
            plots2.append(plot2)

        subplot.set_yscale('log', base=2)
        subplot.set_ylabel(r"$n^{\ast}_{\mathrm{calls}}$")
        subplot.set_xlabel(r"$L$")
        subplot.set_xticks(np.arange(min_qubits, max_qubits+1, 2))

        first_legend = subplot.legend(handles=plots1, title=r"VQE $d=$"+f"{depth1}", loc='upper left', fontsize="x-small")
        ax = subplot.add_artist(first_legend)

        subplot.legend(handles=plots2, loc='lower right', title=r"VQE $d=$"+f"{depth2} Noise", fontsize="x-small")

        subplot.set_title(r"Scaling and fit with $a \cdot 2^{kL}$", fontsize=14)

        subplot.text(14, 2**3.9, '(c)')

plt.savefig("img/fig1.png", dpi=300)


### Inplot


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

qubits = 14
depth = 2
algo = 'vqe'
#xmin, xmax = 2**6, 2**10
xmin, xmax = 25000,520000
ymin, ymax = 0.6, 1.04

for shot, data in ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] == qubits)].groupby('shots'):
    x_axes = data['t'] #*dat[dat['seed'] == 1]["circ_depth"]
    y_axes = data['p_succ']
    ax.plot(x_axes, y_axes, "-o", label=f"{shot}")

ax.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

ax.set_xlabel(r"$n_{calls}$")
ax.set_ylabel(r"$F_{succ}$")

ax.legend(title=r'Shots $M$')

ax.set_title(f"L={qubits} ferro VQE")

plt.savefig("img/inplot.png", dpi=300)


### Supplementary Material


#### Figure A1


In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

fig, ax = plt.subplots()

min_qubits, max_qubits = 4, 18
min_qubits_toplot, max_qubits_toplot = 4, 19
qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)

depth1 = 2
depth2 = 3

algo = 'qaoa' 

ax.plot(np.arange(min_qubits, max_qubits_toplot), fun_exp(np.arange(min_qubits, max_qubits_toplot), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

plots1, plots2 = [], []

p_tresholds = [0.99, 0.9, 0.75, 0.5, 0.25]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for j, (p, color) in enumerate(zip(p_tresholds, colors)):
    data1 = ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth1) & (ferro_df['qubits'] >= min_qubits)]
    data2 = ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth2) & (ferro_df['qubits'] >= min_qubits)]
    
    gt_p1 = data1[data1['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
    gt_p2 = data2[data2['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

    popt1, _ = optimize.curve_fit(fun_lin, gt_p1['qubits'], np.log2(gt_p1['t']))
    plot1, = ax.plot(gt_p1['qubits'], fun_exp(gt_p1['qubits'], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=color)
    plots1.append(plot1)
    plot1, = ax.plot(gt_p1['qubits'], gt_p1['t'], "o", label=r"$F_{succ}$"+f"={p}", color=color)
    plots1.append(plot1)

    popt2, _ = optimize.curve_fit(fun_lin, gt_p2['qubits'], np.log2(gt_p2['t']))
    plot2, = ax.plot(gt_p2['qubits'], fun_exp(gt_p2['qubits'], 2**popt2[0], popt2[1]), "--", label=f"a={2**popt2[0]:.2f}, b={popt2[1]:.2f}", color=colors[j+5])
    plots2.append(plot2)
    plot2, = ax.plot(gt_p2['qubits'], gt_p2['t'], "o", label=r"$F_{succ}$"+f"={p}", fillstyle='none', color=colors[j+5])
    plots2.append(plot2)

ax.set_yscale('log', base=2)
ax.set_ylabel(r"$n_{calls}$")
ax.set_xlabel(r"$L$")
ax.set_xticks(np.arange(min_qubits, max_qubits+1, 2))

first_legend = ax.legend(handles=plots1, title=r"QAOA $p=$"+f"{depth1}", loc='upper left', fontsize="x-small")
ax1 = plt.gca().add_artist(first_legend)
ax.legend(handles=plots2, loc='lower right', title=r"QAOA $p=$"+f"{depth2}", fontsize="xx-small")

ax.set_title(r"Fit with $a \cdot 2^{b \cdot L}$")

L=plt.savefig("img/scalingferrop4.png", dpi=300)


#### Figure A2

In [ ]:
depth = 2
algo = 'qaoa'
max_size = 18

fig = plt.figure(constrained_layout=True, figsize=(22,10))
subplots = fig.subplots(2,4)
for (qubits, data), subplot in zip(ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] <= max_size)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        if ((qubits==4) & (shot > 4)):
            continue 
        if ((qubits==8) & (shot > 32)):
            continue
        # if ((qubits==10) & (shot > 1024)):
        #     continue 
        # if ((qubits==16) & (shot < 16)):
        #     continue 
        # if ((qubits==18) & (shot < 32)):
        #     continue 


        x_axes = dat['t']#*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat['p_succ']

        subplot.plot(x_axes, y_axes, marker="o", label=f"{shot}")
        #subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
    subplot.set_xscale("log", base=2)
    
    subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
    subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

    subplot.set_title(f"L={qubits} Ferro " + r"QAOA $d=$" + f"{depth}", fontsize=14)
    subplot.legend(title=r'Shots $M$')

plt.savefig('img/QAOAsupp.png', dpi=300)


#### Figure A3


In [ ]:
depth = 2
algo = 'vqe'
max_size = 18

fig = plt.figure(constrained_layout=True, figsize=(28,5))
subplots = fig.subplots(1,5)
for (qubits, data), subplot in zip(noise_df[(noise_df['algo'] == algo) & (noise_df['circ_depth'] == depth) & (noise_df['circ_depth'] <= max_size)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        if ((qubits==4) & (shot > 8)):
            continue 
        if ((qubits==6) & (shot > 64)):
            continue
        if ((qubits==8) & (shot > 256)):
            continue
        # if ((qubits==10) & (shot > 1024)):
        #     continue 
        # if ((qubits==16) & (shot < 16)):
        #     continue 
        # if ((qubits==18) & (shot < 32)):
        #     continue 


        x_axes = dat['t']#*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat['p_succ']

        subplot.plot(x_axes, y_axes, marker="o", label=f"{shot}")
        #subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
    subplot.set_xscale("log", base=2)
    
    subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
    subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

    subplot.set_title(f"L={qubits} Ferro " + r"VQE $d=$" + f"{depth} Noise", fontsize=14)
    subplot.legend(title=r'Shots $M$')

plt.savefig('img/VQENoisesupp.png', dpi=300)


#### Figure A4


In [ ]:
depth = 2
algo = 'vqe'
max_size = 18

fig = plt.figure(constrained_layout=True, figsize=(22,10))
subplots = fig.subplots(2,4)
for (qubits, data), subplot in zip(ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['qubits'] <= max_size)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        if ((qubits==4) & (shot > 16)):
            continue 
        if ((qubits==8) & (shot > 256)):
            continue
        if ((qubits==10) & (shot > 1024)):
            continue 
        if ((qubits==16) & (shot < 16)):
            continue 
        if ((qubits==18) & (shot < 32)):
            continue 


        x_axes = dat['t']#*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat['p_succ']

        subplot.plot(x_axes, y_axes, marker="o", label=f"{shot}")
        #subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
    subplot.set_xscale("log", base=2)
    
    subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
    subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

    subplot.set_title(f"L={qubits} Ferro " + r"VQE $d=$" + f"{depth}", fontsize=14)
    subplot.legend(title=r'Shots $M$')

plt.savefig('img/VQEsupp.png', dpi=300)


#### Figure A3


In [ ]:
depth = 2
algo = 'qaoa'
max_size = 18
iter_final = 40
iter_final2 = 30

fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(6,5))
#subplots = fig.subplots(2,4)

#for (iter, data), plt in zip(ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['circ_depth'] <= max_size)].groupby('iter'), subplots.ravel()):
data = ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & ((ferro_df['iter'] == 1) | (ferro_df['iter'] == iter_final) | (ferro_df['iter'] == iter_final2))]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for i, (shot, dat) in enumerate(data.groupby('shots')):
    if shot != 16:
        continue 

    data1 = dat[(dat['iter'] == 1)]
    #data_final = iter1_qaoa
    data_final = dat[(dat['iter'] == iter_final)]
    data_final2 = dat[(dat['iter'] == iter_final2)]



    # x_axes = dat['qubits']#*dat[dat['seed'] == 1]["circ_depth"]
    # y_axes = dat['p_succ']

    popt1, _ = optimize.curve_fit(fun_lin,  data_final['qubits'][-4:], np.log2(data_final['p_succ_final'][-4:]))
    plot1, = ax.plot(data_final['qubits'], data_final['p_succ_final'], "o", label=r"$n_{\mathrm{iter}} =$" + f"{iter_final}", color=colors[i])
    plots1.append(plot1)
    plot1, = ax.plot(data_final['qubits'], fun_exp(data_final['qubits'], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=colors[i])
    plots1.append(plot1)

    popt1bis, _ = optimize.curve_fit(fun_lin,  data_final2['qubits'][-6:], np.log2(data_final2['p_succ_final'][-6:]))
    plot1, = ax.plot(data_final2['qubits'], data_final2['p_succ_final'], "o", label=r"$n_{\mathrm{iter}} =$" + f"{iter_final2}", color=colors[i+1])
    plots1.append(plot1)
    plot1, = ax.plot(data_final2['qubits'][1:], fun_exp(data_final2['qubits'][1:], 2**popt1bis[0], popt1bis[1]), "-", label=f"a={2**popt1bis[0]:.2f}, b={popt1bis[1]:.2f}", color=colors[i+1])
    plots1.append(plot1)

    popt2, _ = optimize.curve_fit(fun_lin, data1['qubits'][-6:], np.log2(data1['p_succ'][-6:]))
    plot2, = ax.plot(data1['qubits'], data1['p_succ'], "o", label=r"$n_{\mathrm{iter}} = 1$", color=colors[i+2])
    plots2.append(plot2)
    plot2, = ax.plot(data1['qubits'][1:], fun_exp(data1['qubits'][1:], 2**popt2[0], popt2[1]), "-", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=colors[i+2])
    plots2.append(plot2)

    #ax.plot(x_axes, y_axes, marker="o", label=f"{shot}")
    #subplot.fill_between(x_axes, data1.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

#ax.plot(np.arange(2, 20, 2), np.exp(-np.arange(2, 20, 2)), '--', color="black")
ax.set_yscale("log", base=10)

ax.set_xlabel(r"$L$")
ax.set_ylabel(r"$F_{\mathrm{succ}}$")

ax.set_title(f"Ferro " + r"QAOA $d=$" + f"{depth} fit with " + r"$a \cdot 2^{kL}$", fontsize=14)
ax.legend(title=r"Shots $M$"+f"=16", loc='lower left', fontsize='small')

plt.savefig('img/QAOA_scaling.png', dpi=300)

In [ ]:
iter1_qaoa = pd.read_csv("data_-pi_iter1.csv")
iter1_qaoa_initcorrect = pd.read_csv("data_0pi_iter1.csv")



depth = 2
algo = 'qaoa'
max_size = 18
iter_final = 40
iter_final2 = 30

fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(6,5))
#subplots = fig.subplots(2,4)

#for (iter, data), plt in zip(ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['circ_depth'] <= max_size)].groupby('iter'), subplots.ravel()):
data = ferro_df[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & ((ferro_df['iter'] == 1) | (ferro_df['iter'] == iter_final) | (ferro_df['iter'] == iter_final2))]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for i, (shot, dat) in enumerate(data.groupby('shots')):
    if shot != 16:
        continue 

    data1 = dat[(dat['iter'] == 1)]
    data_final = iter1_qaoa
    data_final2 = iter1_qaoa_initcorrect
    #data_final = dat[(dat['iter'] == iter_final)]
    #data_final2 = dat[(dat['iter'] == iter_final2)]



    # x_axes = dat['qubits']#*dat[dat['seed'] == 1]["circ_depth"]
    # y_axes = dat['p_succ']

    popt1, _ = optimize.curve_fit(fun_lin,  data_final['qubits'][2:], np.log2(data_final['p_succ_final'][2:]))
    plot1, = ax.plot(data_final['qubits'], data_final['p_succ_final'], "o", label="Correct QAOA", color=colors[i+1])
    plots1.append(plot1)
    plot1, = ax.plot(data_final['qubits'][1:], fun_exp(data_final['qubits'][1:], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=colors[i+1])
    plots1.append(plot1)

    # popt1bis, _ = optimize.curve_fit(fun_lin,  data_final2['qubits'][-6:], np.log2(data_final2['p_succ_final'][-6:]))
    # plot1, = ax.plot(data_final2['qubits'], data_final2['p_succ_final'], "o", label="Correct QAOA " + r"$\theta \in [0,\pi]$", color=colors[i])
    # plots1.append(plot1)
    # plot1, = ax.plot(data_final2['qubits'], fun_exp(data_final2['qubits'], 2**popt1bis[0], popt1bis[1]), "-", label=f"a={2**popt1bis[0]:.2f}, b={popt1bis[1]:.2f}", color=colors[i])
    # plots1.append(plot1)

    popt2, _ = optimize.curve_fit(fun_lin, data1['qubits'][2:], np.log2(data1['p_succ'][2:]))
    plot2, = ax.plot(data1['qubits'], data1['p_succ'], "o", label="Wrong QAOA", color=colors[i+2])
    plots2.append(plot2)
    plot2, = ax.plot(data1['qubits'][1:], fun_exp(data1['qubits'][1:], 2**popt2[0], popt2[1]), "-", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=colors[i+2])
    plots2.append(plot2)

    #ax.plot(x_axes, y_axes, marker="o", label=f"{shot}")
    #subplot.fill_between(x_axes, data1.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

#ax.plot(np.arange(2, 20, 2), np.exp(-np.arange(2, 20, 2)), '--', color="black")
ax.set_yscale("log", base=10)

ax.set_xlabel(r"$L$")
ax.set_ylabel(r"$F_{\mathrm{succ}}$")

ax.set_title(f"Ferro " + r"QAOA $d=$" + f"{depth} fit with " + r"$a \cdot 2^{kL}$", fontsize=14)
ax.legend(title=r"Shots $M$"+f"=16", loc='lower left', fontsize='small')

plt.savefig('img/QAOA_iter1.png', dpi=300)



#### Figure A5

Linear initialization as in [Quantum annealing initialization of the quantum approximate optimization algorithm](http://arxiv.org/abs/2101.05742).

In [ ]:
opt_qaoa = pd.read_csv("data-opt.csv", index_col=0)
opt_qaoa_2 = pd.read_csv("data-opt2.csv", index_col=0)

In [ ]:
opt_qaoa_2.head()

In [ ]:
depth = 2
algo = 'qaoa'

fig = plt.figure(constrained_layout=True, figsize=(22,10))
subplots = fig.subplots(2,4)
for (qubits, data), subplot in zip(opt_qaoa[(opt_qaoa['algo'] == algo) & (opt_qaoa['circ_depth'] == depth)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        # if ((qubits==4) & (shot > 4)):
        #     continue 
        # if ((qubits==8) & (shot > 32)):
        #     continue
        # if ((qubits==10) & (shot > 1024)):
        #     continue 
        # if ((qubits==16) & (shot < 16)):
        #     continue 
        # if ((qubits==18) & (shot < 32)):
        #     continue 

        x_axes = dat['t']
        y_axes = dat['p_succ']

        subplot.plot(x_axes, y_axes, marker="o", label=f"{shot}")
        #subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
    subplot.set_xscale("log", base=2)
    
    subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
    subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

    subplot.set_title(f"L={qubits} Ferro " + r"QAOA $d=$" + f"{depth}", fontsize=14)
    subplot.legend(title=r'Shots $M$')

plt.savefig('img/QAOA_opt.png', dpi=300)


#### Figure A6


In [ ]:
depth = 2
algo = 'qaoa'
max_size = 18

fig = plt.figure(constrained_layout=True, figsize=(22,10))
subplots = fig.subplots(2,4)
for (qubits, data), subplot in zip(opt_qaoa_2[(opt_qaoa_2['algo'] == algo) & (opt_qaoa_2['circ_depth'] == depth) & (opt_qaoa_2['qubits'] <= max_size)].groupby('qubits'), subplots.ravel()):
    for shot, dat in data.groupby('shots'):
        # if ((qubits==4) & (shot > 4)):
        #     continue 
        # if ((qubits==8) & (shot > 32)):
        #     continue
        # if ((qubits==10) & (shot > 1024)):
        #     continue 
        # if ((qubits==16) & (shot < 16)):
        #     continue 
        # if ((qubits==18) & (shot < 32)):
        #     continue 


        x_axes = dat['t']#*dat[dat['seed'] == 1]["circ_depth"]
        y_axes = dat['p_succ']

        subplot.plot(x_axes, y_axes, marker="o", label=f"{shot}")
        #subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black")
    subplot.set_xscale("log", base=2)
    
    subplot.set_xlabel(r"$n_{\mathrm{calls}}$")
    subplot.set_ylabel(r"$F_{\mathrm{succ}}$")

    subplot.set_title(f"L={qubits} Ferro " + r"QAOA $d=$" + f"{depth}", fontsize=14)
    subplot.legend(title=r'Shots $M$')

plt.savefig('img/QAOA_opt2.png', dpi=300)


#### Figure A7


In [ ]:
def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

fig, ax = plt.subplots()

min_qubits, max_qubits = 4, 16
min_qubits_toplot, max_qubits_toplot = 4, 12
qubits_toplot = np.arange(min_qubits_toplot, max_qubits_toplot)

depth1 = 2
depth2 = 2

algo = 'qaoa' 

ax.plot(np.arange(min_qubits, max_qubits_toplot), fun_exp(np.arange(min_qubits, max_qubits_toplot), 1, 1), color='red', linewidth=3.5, linestyle="dashed")

plots1, plots2 = [], []

p_tresholds = [0.9, 0.75, 0.5, 0.25]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for j, (p, color) in enumerate(zip(p_tresholds, colors)):
    data1 = opt_qaoa[(opt_qaoa['algo'] == algo) & (opt_qaoa['circ_depth'] == depth1) & (opt_qaoa['qubits'] >= min_qubits) & (opt_qaoa['qubits'] <= max_qubits)]
    data2 = opt_qaoa_2[(opt_qaoa_2['algo'] == algo) & (opt_qaoa_2['circ_depth'] == depth2) & (opt_qaoa_2['qubits'] >= min_qubits) & (opt_qaoa_2['qubits'] <= max_qubits)]
    
    gt_p1 = data1[data1['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()
    gt_p2 = data2[data2['p_succ'] > p].groupby('qubits', as_index=False)['t'].min()

    popt1, _ = optimize.curve_fit(fun_lin, gt_p1['qubits'], np.log2(gt_p1['t']))
    plot1, = ax.plot(gt_p1['qubits'], fun_exp(gt_p1['qubits'], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=color)
    plots1.append(plot1)
    plot1, = ax.plot(gt_p1['qubits'], gt_p1['t'], "o", label=r"$F_{succ}$"+f"={p}", color=color)
    plots1.append(plot1)

    popt2, _ = optimize.curve_fit(fun_lin, gt_p2['qubits'], np.log2(gt_p2['t']))
    plot2, = ax.plot(gt_p2['qubits'], fun_exp(gt_p2['qubits'], 2**popt2[0], popt2[1]), "--", label=f"a={2**popt2[0]:.2f}, b={popt2[1]:.2f}", color=colors[j+5])
    plots2.append(plot2)
    plot2, = ax.plot(gt_p2['qubits'], gt_p2['t'], "o", label=r"$F_{succ}$"+f"={p}", fillstyle='none', color=colors[j+5])
    plots2.append(plot2)

ax.set_yscale('log', base=2)
ax.set_ylabel(r"$n_{calls}$")
ax.set_xlabel(r"$L$")
ax.set_xticks(np.arange(min_qubits, max_qubits+1, 2))

first_legend = ax.legend(handles=plots1, title=r"QAOA Linear Opt.", loc='upper left', fontsize="xx-small")
ax1 = plt.gca().add_artist(first_legend)
ax.legend(handles=plots2, loc='lower right', title=r"QAOA SV Opt", fontsize="xx-small")

ax.set_title(r"Fit with $a \cdot 2^{b \cdot L}$")

plt.savefig("img/scalingferro_opt.png", dpi=300)


#### Figure A8

Plot with fixed M=16 (as Guglielmo's)

In [ ]:
opt_qaoa.describe()

In [ ]:
depth = 2
algo = 'qaoa'
max_size = 16
iter_final = 40
iter_final2 = 30

fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(6,5))
#subplots = fig.subplots(2,4)

#for (iter, data), plt in zip(opt_qaoa[(ferro_df['algo'] == algo) & (ferro_df['circ_depth'] == depth) & (ferro_df['circ_depth'] <= max_size)].groupby('iter'), subplots.ravel()):
data1 = opt_qaoa[(opt_qaoa['algo'] == algo) & (opt_qaoa['circ_depth'] == depth) & (opt_qaoa['iter'] == 1) & (opt_qaoa['qubits'] <= max_size)] # | (opt_qaoa['iter'] == iter_final) | (opt_qaoa['iter'] == iter_final2))]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for i, (shot, dat) in enumerate(data1.groupby('shots')):
    if shot != 16:
        continue 
    #data_final = dat[(dat['iter'] == iter_final)]
    #data_final2 = dat[(dat['iter'] == iter_final2)]

    # x_axes = dat['qubits']#*dat[dat['seed'] == 1]["circ_depth"]
    # y_axes = dat['p_succ']

    popt1, _ = optimize.curve_fit(fun_lin,  dat['qubits'][], np.log2(dat['p_succ_final']))
    plot1, = ax.plot(dat['qubits'], dat['p_succ_final'], "o", label="Linear init", color=colors[i+1])
    plots1.append(plot1)
    plot1, = ax.plot(dat['qubits'], fun_exp(dat['qubits'], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=colors[i+1])
    plots1.append(plot1)

data1_final = opt_qaoa[(opt_qaoa['algo'] == algo) & (opt_qaoa['circ_depth'] == depth) & (opt_qaoa['iter'] == iter_final) & (opt_qaoa['qubits'] <= max_size)] # | (opt_qaoa['iter'] == iter_final) | (opt_qaoa['iter'] == iter_final2))]
color = 'r'
for i, (shot, dat) in enumerate(data1_final.groupby('shots')):
    if shot != 16:
        continue 
    #data_final = dat[(dat['iter'] == iter_final)]
    #data_final2 = dat[(dat['iter'] == iter_final2)]

    # x_axes = dat['qubits']#*dat[dat['seed'] == 1]["circ_depth"]
    # y_axes = dat['p_succ']

    popt1, _ = optimize.curve_fit(fun_lin,  dat['qubits'], np.log2(dat['p_succ_final']))
    plot1, = ax.plot(dat['qubits'], dat['p_succ_final'], "+", label="Linear init, Noise opt (40 steps)", color=color)
    plots1.append(plot1)
    plot1, = ax.plot(dat['qubits'][1:], fun_exp(dat['qubits'][1:], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=color)
    plots1.append(plot1)

data2 = opt_qaoa_2[(opt_qaoa_2['algo'] == algo) & (opt_qaoa_2['circ_depth'] == depth) & (opt_qaoa_2['iter'] == 1)] # | (opt_qaoa_2['iter'] == iter_final) | (opt_qaoa_2['iter'] == iter_final2))]
colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for i, (shot, dat) in enumerate(data2.groupby('shots')):
    if shot != 16:
        continue 

    popt2, _ = optimize.curve_fit(fun_lin, dat['qubits'][2:], np.log2(dat['p_succ'][2:]))
    plot2, = ax.plot(dat['qubits'], dat['p_succ'], "x", label="After SV Opt", color=colors[i+2])
    plots2.append(plot2)
    plot2, = ax.plot(dat['qubits'][1:], fun_exp(dat['qubits'][1:], 2**popt2[0], popt2[1]), "-", label=f"a={2**popt2[0]:.2f}, k={popt2[1]:.2f}", color=colors[i+2])
    plots2.append(plot2)

    #ax.plot(x_axes, y_axes, marker="o", label=f"{shot}")
    #subplot.fill_between(x_axes, data2.groupby(['iter'])['p_succ'].quantile(0.95), dat.groupby(['iter'])['p_succ'].quantile(0.05), alpha=0.3)

#ax.plot(np.arange(2, 20, 2), np.exp(-np.arange(2, 20, 2)), '--', color="black")
ax.set_yscale("log", base=10)

ax.grid(linewidth = 0.2)

ax.set_xlabel(r"$L$")
ax.set_ylabel(r"$F_{\mathrm{succ}}$")

ax.set_title(f"Ferro " + r"QAOA $d=2$", fontsize=14)
ax.legend(title=r"Shots $M$"+f"=16", loc='lower left', fontsize='small')

plt.savefig("img/m16-smart_init.png", dpi=300)




#### Figure A9



In [ ]:
data_opt_080 = pd.read_csv("data_opt16shots.csv", index_col=0)

In [ ]:
data_opt_080.describe()

In [ ]:
depths = [2,4,8,12,16,20,24,28,32,36]
algo = 'qaoa'

def fun_exp(x, a, b):
    return a*2**(b*x)

def fun_lin(x, a, b):
    return a + b*x

fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(6,5))

colors = ['lightgreen', 'limegreen', 'forestgreen', 'darkgreen','lightseagreen', 'skyblue', 'cornflowerblue' , 'royalblue', 'mediumblue', 'darkblue']
for k, depth in enumerate(depths):
    if ((depth == 2) or (depth == 4) or (depth == 8) or (depth == 16) or (depth == 32)):
        dat = data_opt_080[(data_opt_080['algo'] == algo) & (data_opt_080['circ_depth'] == depth)]
        dat = dat[dat["p_succ_final"]!=0]

        popt1, _ = optimize.curve_fit(fun_lin,  dat['qubits'][2:], np.log2(dat['p_succ_final'][2:]))
        ax.plot(dat['qubits'], dat['p_succ_final'], "o", label=f"d={depth}", color=colors[k+1])
        ax.plot(dat['qubits'][2:], fun_exp(dat['qubits'][2:], 2**popt1[0], popt1[1]), "-", label=f"a={2**popt1[0]:.2f}, b={popt1[1]:.2f}", color=colors[k+1])


ax.set_xticks(np.arange(data_opt_080['qubits'].min(), data_opt_080['qubits'].max() + 2, 2))
ax.set_yscale("log", base=10)

ax.grid(linewidth = 0.2)

ax.set_xlabel(r"$L$")
ax.set_ylabel(r"$F_{\mathrm{succ}}$")

ax.set_title(f"Linear init " + r" Fit $a \cdot 2^{bx}$", fontsize=14)
ax.legend(title=r"Shots $M$"+f"=16 " + " no opt", loc='lower left', fontsize='small')

plt.savefig("img/depth-smart_init.png", dpi=300)


### Scaling computation


In [ ]:


L = 100
# depth
d = 2
#seconds
t_gate = 10**(-8) 
# exponent
k = 0.5

print(f"Month {2**(k*L) * d * t_gate / 60 / 60 / 24 / 30}") # months


### Figure A2
Energy per sample increasing L


In [ ]:
spins = 8
dim = 1
h_ext = 0.05
type_ising = "random"
seed = 42 # not used

J, h = utils.get_ising_params(spins, h_ext, type_ising, seed)
print(J, h)

adj_dict = {}
for i in range(spins):
    if i == spins - 1:
        continue
    adj_dict[(i, i + 1)] = J[i]
# class devoted to set the couplings and get the energy
model = ising.Ising(spins, dim=dim, adj_dict=adj_dict, h_field=h)

energies = []
for i, sample in enumerate(itertools.product([0, 1], repeat=model.spins)):
    print(sample)
    energies.append(model.energy(np.asarray(sample) * 2 - 1))

In [ ]:
plt.plot(energies)

In [ ]:
# Select length of axes and the space between tick labels
xmin, xmax, ymin, ymax = 0, 265, -8, 8

# Plot points
fig, ax = plt.subplots(figsize=(8, 4))


# Draw lines connecting points to axes
line1, = ax.plot(np.arange(5, len(energies)+5), energies, color='black', linewidth=1.5)

# Set identical scales for both axes
ax.set(xlim=(xmin-1, xmax+1), ylim=(ymin-1, ymax+1))

# Set bottom and left spines as x and y axes of coordinate system
ax.spines['bottom'].set_position('zero')
ax.spines['left'].set_position('zero')

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create 'x' and 'y' labels placed at the end of the axes
ax.set_xlabel('L', size=18, labelpad=-24, x=1.03)
ax.set_ylabel('Energy', size=18, labelpad=-21, y=1.02, rotation=0)

# Draw arrows
arrow_fmt = dict(markersize=4, color='black', clip_on=False)
ax.plot((1), (0), marker='>', transform=ax.get_yaxis_transform(), **arrow_fmt)
ax.plot((0), (1), marker='^', transform=ax.get_xaxis_transform(), **arrow_fmt)

ax.tick_params(  
    axis='both',   # changes apply to both axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left = True,
    right=False,
    labelleft=True,
    labelbottom=False,
)

plt.savefig("figa2.png", dpi=300)


## Ferromagnetic model



In [ ]:
import itertools
import os

import numpy as np
import pandas as pd
import json
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
sizes = [4,6,8,10,12,14,16,18,20]

ferro_all = []
for size in sizes:
    ferro_all.append(pd.read_csv(f"data_n{size}-test.csv"))

In [ ]:
ferro_df = pd.concat(ferro_all, ignore_index=True)
ferro_df .to_csv(f"data_ferro-UNION.csv", index=False)

In [ ]:
depth = 2

fig = plt.figure(constrained_layout=True, figsize=(16,10))
subplots = fig.subplots(2,3)
for (qubits, data), subplot in zip(ferro_df[(ferro_df['algo'] == 'vqe') & (ferro_df['circ_depth'] == depth)].groupby('qubits'), subplots.ravel()):
    print(qubits)
    # for shot, dat in data.groupby('shots'):
    #     x_axes = dat['t']
    #     y_axes = dat['p_succ']

    # subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black", label="Random sampling")
    # subplot.set_xscale("log", base=2)

    # subplot.set_xlabel(r"$n_{calls}$")
    # subplot.set_ylabel(r"$F('success')$")

    # subplot.set_title(f"N={qubits} VQE p={depth}")
    # subplot.legend(title='Shots')

In [ ]:
ferro_df['qubits']

In [ ]:
prendere varianza dei parametri e mettila come errorbar
cambiare nome del QAOA+ con meno oppure like
attenzione a qaoa, inserire Rz (ext_field)
g100-scratch path dove salvare


## Process raw data


In [ ]:
import itertools
import os
import math

import numpy as np
import pandas as pd
import json
from scipy import optimize
import matplotlib.pyplot as plt

In [ ]:
dims = np.arange(4,20,2)
circ_depths = [2]
algs = ["qaoa"]
seeds = np.arange(1, 31)


model_info = [(n, algo, p, seed) for p in circ_depths for algo in algs for n in dims for seed in seeds] 

In [ ]:
# NOT TO USE, SEE BELOW
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": [], "seed": []}
for (n, algo, p, seed) in model_info:
    dir_path = f"results/N{n}/{algo}p{p}_{seed}/"
    if not os.path.isdir(dir_path):
        #print(f"{dir_path} does not exist")
        continue
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # data are a list with #initial_points elements
            runs = json.load(file)
        ever_founds = []
        iterations = [] 
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for r in runs:
            ever_founds.append(bool(r["ever_found"]))
            iterations.append(r["nfev"])
        # compute p('found minimum')
        #run_info["p_succ"].append(ever_founds)
        run_info["p_succ"].append(
            np.mean(ever_founds, dtype=np.float128 )
        )  # float128 to avoid to many zeros
        run_info["iter"].append(int(math.ceil(np.mean(iterations))))
        # maxiter*shots = actual number of iteration
        run_info["t"].append(r["shots"] * run_info["iter"][-1])
        run_info["shots"].append(r["shots"])
        run_info["qubits"].append(n)
        run_info["circ_depth"].append(p)
        run_info["algo"].append(algo)
        run_info["seed"].append(seed)
        
jobs_df = pd.DataFrame(run_info)
jobs_df.to_csv(f"data_n14-12_PART.csv")

In [ ]:
dims = np.arange(4,24,2)
circ_depths = np.arange(2,38,2)
algs = ["qaoa"]
#seeds = np.arange(1, 31) 

model_info = [(n, algo, p) for p in circ_depths for algo in algs for n in dims] 

In [ ]:
run_info = {"t": [], "shots": [], "iter": [], "p_succ": [], "qubits": [], "circ_depth": [], "algo": [], "succ_true": [], "succ_false": [], 'p_succ_final': []}
for (n, algo, p) in model_info:
    dir_path = f"results_opt2/{algo}p{p}_n{n}/"
    if not os.path.isdir(dir_path):
        print(f"{dir_path} does not exist")
        continue
    for filename in sorted(os.listdir(dir_path)):
        filename = dir_path + filename
        with open(filename, "r") as file:
            # data are a list with #initial_points elements
            runs = json.load(file)
        ever_founds = []
        success_final = []
        iterations = [] 
        # for each shot and iteration param
        # we randomized the initial point
        # to estimate the right probability
        for r in runs:
            ever_founds.append(r["ever_found"])
            iterations.append(r["nfev"])
            success_final.append(r["success"])
        # compute p('found minimum')
        #run_info["p_succ"].append([ever_founds])
        run_info["p_succ"].append(
            np.mean(np.asarray(ever_founds, dtype=np.float128))
        )  # float128 to avoid to many zeros
        run_info["p_succ_final"].append(
            np.mean(np.asarray(success_final, dtype=np.float128))
        )  # float128 to avoid to many zeros
        run_info["succ_true"].append(np.count_nonzero(ever_founds))
        run_info["succ_false"].append(len(ever_founds) - np.count_nonzero(ever_founds))

        run_info["iter"].append(int(math.ceil(np.mean(np.asarray(iterations)))))
        # maxiter*shots = actual number of iteration
        run_info["t"].append(int(r["shots"] * run_info["iter"][-1]))
        run_info["shots"].append(int(r["shots"]))
        run_info["qubits"].append(int(n))
        run_info["circ_depth"].append(int(p))
        run_info["algo"].append(str(algo))
        
jobdf = pd.DataFrame.from_dict(run_info)
jobdf.to_csv(f"data_opt16shots.csv")

In [ ]:
print(pd.load_csv("data-opt.csv"))

In [ ]:
jobs_correct = pd.read_csv("data_n14-12_PART.csv")

fig = plt.figure(constrained_layout=True, figsize=(12,5))
subplots = fig.subplots(1,2)
for (qubits, data), subplot in zip(jobs_correct[(jobs_correct['algo'] == 'qaoa') & (jobs_correct['circ_depth'] == 1) & (jobs_correct['iter'] < 25)].groupby('qubits'), subplots):
    for shot, dat in data.groupby('shots'):
        x_axes = dat[dat['seed'] == 1]['t']*dat[dat['seed'] == 1]['circ_depth'].mean()
        y_axes = dat.groupby(['iter'])['p_succ'].median()
        error = [
            (dat.groupby(['iter'])['p_succ'].median() - dat.groupby(['iter'])['p_succ'].quantile(0.25)).tolist(),
            (dat.groupby(['iter'])['p_succ'].quantile(0.75) - dat.groupby(['iter'])['p_succ'].median()).tolist(),
            ]
        subplot.errorbar(x_axes, y_axes, yerr=error, marker="o", label=f"{shot}")
        subplot.fill_between(x_axes, dat.groupby(['iter'])['p_succ'].min(), dat.groupby(['iter'])['p_succ'].max(), alpha=0.3)

    subplot.plot(np.arange(0,2**(5+qubits),2), 1 - (1-1/(2**qubits))**np.arange(0,2**(5+qubits),2), '--', color="black", label="Random sampling")
    subplot.set_xscale("log", base=2)

    subplot.set_xlabel("iterations x shots x depth")
    subplot.set_ylabel(r"$F('success')$")

    subplot.set_title(f"N={qubits} QAOA p={int(dat[dat['seed'] == 1]['circ_depth'].mean())}")
    subplot.legend(title='Shots')

plt.savefig('spinglass_p1.png')